쿼리 정제할때 영어+한글 2가지로 질문 생성

하이드 가상 답변 만들때도 영어+한글 2가지로 답변 생성

dense로만 하고, 재시도 할때 원문 10개 / QA 40개

최종 답변할때, 검색 문서로 처음 쿼리로 검색한 거 & 두 번쨰 재시도할때 검색한거 (각각 QA, 원문 해서 총 4가지로 제공)

In [1]:
from pathlib import Path
import sys

root = Path.cwd()
while not (root / "utils_v6").exists() and root != root.parent:
    root = root.parent
sys.path.insert(0, str(root))

from utils_v6.main3 import (run_langraph)

c:\Users\playdata2\miniconda3\envs\final-clean\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Workspaces\SKN14-Final-1Team-AI\LLM_hybrid\utils_v6\retriever.py:51: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vs = Chroma(


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
# 필요한 라이브러리 임포트
import os
import pandas as pd
from pprint import pprint
# LangChain 관련 라이브러리
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# RAGAS 관련 라이브러리
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.persona import Persona
from ragas.testset import TestsetGenerator
from ragas import EvaluationDataset, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

In [3]:
import pandas as pd

# 같은 폴더(현재 작업 디렉토리)에 저장했다면
dataset_df = pd.read_csv("./dataset2.csv")
print(dataset_df.shape)
display(dataset_df)

(20, 4)


,user_input,reference_contexts,reference,synthesizer_name
0,CSV 파일을 이용해서 S/MIME 인증서를 관리하려면 어떤 작업이 필요한가요?,['Q: S/MIME 인증서를 관리하기 위해 CSV 파일을 처리하는 함수에서 어떤 ...,"CSV 파일을 처리하는 함수에서는 사용자의 인증서를 나열하고, 기본 인증서가 만료되...",single_hop_specific_query_synthesizer
1,클라우드 파이어스토어 부하 분산기 어떻게 써야되나요?,['Q: Cloud Firestore의 부하 분산 기능은 어떻게 작동하나요?\nA:...,Cloud Firestore는 데이터베이스에 대한 트래픽을 지원하기 위해 자동으로 ...,single_hop_specific_query_synthesizer
2,범위 많으면 동의 어렵나요?,['Q: OAuth 2.0을 사용할 때 요청하는 범위의 수가 사용자 동의에 미치는 ...,요청된 범위 수가 많으면 사용자 동의를 얻을 가능성이 낮아질 수 있습니다.,single_hop_specific_query_synthesizer
3,As a beginner who is not familiar with Google ...,"['Q: Google Cloud 세션 제어 기능이란 무엇이며, 어떤 영향을 미치나요...",When Google Cloud session control features are...,single_hop_specific_query_synthesizer
4,"firebase_functions 어떻게 쓰는지 잘 모르겠는데, Cloud Fire...",['Q: Cloud Firestore에서 특정 문서의 모든 변경에 대해 이벤트를 트...,Cloud Firestore에서 특정 문서의 모든 변경에 이벤트를 트리거하려면 on...,single_hop_specific_query_synthesizer
5,Firebase에서 다단계 인증(MFA)을 사용할 때 Objective-C로 사용자...,['Q: 다단계 인증(MFA) 사용 시 Firebase에서 사용자 인증을 완료하는 ...,다단계 인증(MFA) 사용 시 Firebase에서 사용자 인증을 완료하는 방법은 F...,single_hop_specific_query_synthesizer
6,Java를 사용해서 Firestore에 문서를 추가할 때 어떤 코드를 사용하나요?,['Q: Firestore에서 문서에 데이터를 추가할 때 사용하는 Java 코드 예...,다음은 Java에서 Firestore에 문서를 추가할 때 사용하는 코드 예시입니다....,single_hop_specific_query_synthesizer
7,COLUMNS가 Dimension API에서 무슨 뜻인가요?,['Q: Dimension API에서 사용할 수 있는 열거형 값은 무엇인가요?\nA...,COLUMNS는 Dimension API에서 시트의 열에서 작동함을 의미합니다.,single_hop_specific_query_synthesizer
8,ttll이 뭐에요? 그리고 한도 같은거 있나요?,['Q: TTL 정책을 적용할 때 한도 및 제약조건은 무엇인가요?\nA: 컬렉션 그...,"컬렉션 그룹당 하나의 필드만 TTL 필드로 표시할 수 있고, 필드 수준 구성은 최대...",single_hop_specific_query_synthesizer
9,Me want know what FUNCTION_ENTRYPOINT do in gc...,['Q: gcloud CLI를 사용하여 Cloud Run 함수를 배포한 후 트리거를...,FUNCTION_ENTRYPOINT는 gcloud run deploy 명령어에서 함...,single_hop_specific_query_synthesizer


In [4]:

## 02.RAG 체인 구성

import pandas as pd
from datasets import Dataset
import uuid
import time


evaluated_dataset = []

for _, row in dataset_df.iterrows():
    time.sleep(3)
    query = row.user_input  # 사용자 입력 - 합성 데이터셋에서 가져옴

    # run_langraph 함수 호출
    rag_result = run_langraph(query,'1', None)

    evaluated_dataset.append(
        {
            "user_input": query,
            # "retrieved_contexts": [doc for doc in rag_result["search_results"]],
            "retrieved_contexts": rag_result.get("search_results", []), # basic, none
            "response": rag_result["answer"],
            "reference": row['reference']
        }
    )

print("평가 데이터셋 준비 완료")

# RAGAS 평가 데이터셋 생성
v7_evaluated_dataset = Dataset.from_list(evaluated_dataset)
# csv 데이터 저장
v7_evaluated_dataset.to_pandas().to_csv('[5.utils_v6]ragas_evaluated_dataset_hybrid4.csv', index=False)
v7_evaluated_dataset.to_pandas().info()

run_langraph 호출 - 입력: CSV 파일을 이용해서 S/MIME 인증서를 관리하려면 어떤 작업이 필요한가요?, 이미지: False
analyze_image 호출됨 - 이미지 존재: False
[tool_based_search_node] 실행 - queries=['CSV 파일을 이용한 S/MIME 인증서 관리 방법은 무엇인가요?']
[vector_search_tool] 검색 중: 'CSV 파일을 이용한 S/MIME 인증서 관리 방법은 무엇인가요?' with filters: {'tags': {'$in': ['gmail']}}
[tool_based_search_node] 실행 - state['search_results']=['except (OSError, IOError) as error:\n    print(f"An error occured while reading the CSV file: {error}")\n\n\nif __name__ == "__main__":\n  insert_cert_from_csv(csv_filename="xyz")\n인증서 관리\n이 예에서는 smimeInfo API의 여러 호출을 결합하여 조직의 인증서를 관리하는 방법을 보여줍니다. 사용자의 인증서를 나열하고 기본 인증서가 만료되었거나 설정되지 않은 경우 지정된 파일에서 찾은 인증서를 업로드합니다. 그런 다음 만료일이 가장 늦은 인증서를 기본값으로 설정합니다.\n그런 다음 이전 예와 같이 CSV 파일을 처리하는 함수에서 이를 호출합니다.\n--- 탭: 자바 ---\n.filepath {\n    color: #fff;\n    margin: 6px;\n    max-width: calc(100% - 160px); /* Give at least 160px for the "View on GitHub" button. */\n    text-overflow: ellipsis;\n    text-shadow: rgba(0,0,0,0.1) 1px 1px;\n    overflow: hid

In [5]:
import pandas as pd

v7_evaluated_dataset = pd.read_csv('[5.utils_v6]ragas_evaluated_dataset_hybrid4.csv')

In [6]:
v7_evaluated_dataset.head()

,user_input,retrieved_contexts,response,reference
0,CSV 파일을 이용해서 S/MIME 인증서를 관리하려면 어떤 작업이 필요한가요?,"['except (OSError, IOError) as error:\n pri...",CSV 파일을 이용해서 S/MIME 인증서를 관리하려면 다음과 같은 작업이 필요합니...,"CSV 파일을 처리하는 함수에서는 사용자의 인증서를 나열하고, 기본 인증서가 만료되..."
1,클라우드 파이어스토어 부하 분산기 어떻게 써야되나요?,['Cloud Firestore는 비즈니스 요구사항을 충족하기 위해 자동 확장을 제...,Cloud Firestore는 데이터베이스에 대한 트래픽을 지원하기 위해 자동으로 ...,Cloud Firestore는 데이터베이스에 대한 트래픽을 지원하기 위해 자동으로 ...
2,범위 많으면 동의 어렵나요?,['Cloud Firestore는 비즈니스 요구사항을 충족하기 위해 자동 확장을 제...,"""범위 많으면 동의 어렵나요?""는 제가 모르는 내용입니다. 일상 질문 혹은 구글 A...",요청된 범위 수가 많으면 사용자 동의를 얻을 가능성이 낮아질 수 있습니다.
3,As a beginner who is not familiar with Google ...,['또한 사용자 계정 또는 서비스 계정이 모든 클라이언트에서 보유할 수 있는 총 새...,When Google Cloud session control features are...,When Google Cloud session control features are...
4,"firebase_functions 어떻게 쓰는지 잘 모르겠는데, Cloud Fire...",['문서의 모든 변경사항에 대해 함수 트리거\n\n실행되는 이벤트 유형에 관계없이 ...,Cloud Firestore에서 문서가 변경될 때마다 자동으로 실행되는 기능을 설정...,Cloud Firestore에서 특정 문서의 모든 변경에 이벤트를 트리거하려면 on...


In [7]:
# csv 데이터 저장
v7_evaluated_dataset

,user_input,retrieved_contexts,response,reference
0,CSV 파일을 이용해서 S/MIME 인증서를 관리하려면 어떤 작업이 필요한가요?,"['except (OSError, IOError) as error:\n pri...",CSV 파일을 이용해서 S/MIME 인증서를 관리하려면 다음과 같은 작업이 필요합니...,"CSV 파일을 처리하는 함수에서는 사용자의 인증서를 나열하고, 기본 인증서가 만료되..."
1,클라우드 파이어스토어 부하 분산기 어떻게 써야되나요?,['Cloud Firestore는 비즈니스 요구사항을 충족하기 위해 자동 확장을 제...,Cloud Firestore는 데이터베이스에 대한 트래픽을 지원하기 위해 자동으로 ...,Cloud Firestore는 데이터베이스에 대한 트래픽을 지원하기 위해 자동으로 ...
2,범위 많으면 동의 어렵나요?,['Cloud Firestore는 비즈니스 요구사항을 충족하기 위해 자동 확장을 제...,"""범위 많으면 동의 어렵나요?""는 제가 모르는 내용입니다. 일상 질문 혹은 구글 A...",요청된 범위 수가 많으면 사용자 동의를 얻을 가능성이 낮아질 수 있습니다.
3,As a beginner who is not familiar with Google ...,['또한 사용자 계정 또는 서비스 계정이 모든 클라이언트에서 보유할 수 있는 총 새...,When Google Cloud session control features are...,When Google Cloud session control features are...
4,"firebase_functions 어떻게 쓰는지 잘 모르겠는데, Cloud Fire...",['문서의 모든 변경사항에 대해 함수 트리거\n\n실행되는 이벤트 유형에 관계없이 ...,Cloud Firestore에서 문서가 변경될 때마다 자동으로 실행되는 기능을 설정...,Cloud Firestore에서 특정 문서의 모든 변경에 이벤트를 트리거하려면 on...
5,Firebase에서 다단계 인증(MFA)을 사용할 때 Objective-C로 사용자...,['--- 탭: Objective-C [https://firebase.google....,Objective-C에서 Firebase의 다단계 인증(MFA)을 사용할 때 사용자...,다단계 인증(MFA) 사용 시 Firebase에서 사용자 인증을 완료하는 방법은 F...
6,Java를 사용해서 Firestore에 문서를 추가할 때 어떤 코드를 사용하나요?,['Firestore는 컬렉션에 저장되는 문서에 데이터를 저장합니다.\n문서에 데이...,Java를 사용하여 Firestore에 문서를 추가할 때는 다음과 같은 코드를 사용...,다음은 Java에서 Firestore에 문서를 추가할 때 사용하는 코드 예시입니다....
7,COLUMNS가 Dimension API에서 무슨 뜻인가요?,['COLUMN_NAME: 임베딩 데이터가 포함된 열의 이름입니다. 이 열은 ARR...,COLUMNS는 Dimension API에서 시트의 열에서 작동하는 것을 의미합니다...,COLUMNS는 Dimension API에서 시트의 열에서 작동함을 의미합니다.
8,ttll이 뭐에요? 그리고 한도 같은거 있나요?,['COLUMN_NAME: 임베딩 데이터가 포함된 열의 이름입니다. 이 열은 ARR...,"""ttll과 한도에 관한 질문""은 제가 모르는 내용입니다. 일상 질문 혹은 구글 A...","컬렉션 그룹당 하나의 필드만 TTL 필드로 표시할 수 있고, 필드 수준 구성은 최대..."
9,Me want know what FUNCTION_ENTRYPOINT do in gc...,"[""test_event_stream(auth_token, 'Test token re...","죄송하지만, ""FUNCTION_ENTRYPOINT""에 대한 정보는 제공된 문서에 포...",FUNCTION_ENTRYPOINT는 gcloud run deploy 명령어에서 함...


In [8]:
v7_evaluated_dataset.head()

,user_input,retrieved_contexts,response,reference
0,CSV 파일을 이용해서 S/MIME 인증서를 관리하려면 어떤 작업이 필요한가요?,"['except (OSError, IOError) as error:\n pri...",CSV 파일을 이용해서 S/MIME 인증서를 관리하려면 다음과 같은 작업이 필요합니...,"CSV 파일을 처리하는 함수에서는 사용자의 인증서를 나열하고, 기본 인증서가 만료되..."
1,클라우드 파이어스토어 부하 분산기 어떻게 써야되나요?,['Cloud Firestore는 비즈니스 요구사항을 충족하기 위해 자동 확장을 제...,Cloud Firestore는 데이터베이스에 대한 트래픽을 지원하기 위해 자동으로 ...,Cloud Firestore는 데이터베이스에 대한 트래픽을 지원하기 위해 자동으로 ...
2,범위 많으면 동의 어렵나요?,['Cloud Firestore는 비즈니스 요구사항을 충족하기 위해 자동 확장을 제...,"""범위 많으면 동의 어렵나요?""는 제가 모르는 내용입니다. 일상 질문 혹은 구글 A...",요청된 범위 수가 많으면 사용자 동의를 얻을 가능성이 낮아질 수 있습니다.
3,As a beginner who is not familiar with Google ...,['또한 사용자 계정 또는 서비스 계정이 모든 클라이언트에서 보유할 수 있는 총 새...,When Google Cloud session control features are...,When Google Cloud session control features are...
4,"firebase_functions 어떻게 쓰는지 잘 모르겠는데, Cloud Fire...",['문서의 모든 변경사항에 대해 함수 트리거\n\n실행되는 이벤트 유형에 관계없이 ...,Cloud Firestore에서 문서가 변경될 때마다 자동으로 실행되는 기능을 설정...,Cloud Firestore에서 특정 문서의 모든 변경에 이벤트를 트리거하려면 on...


# 03. 개별 답변 평가
- results_df:
	- 다음 4가지 평가 기준으로 1-5점 척도로 점수 평가
	1. 응답 정확성 (Answer Correctness) : 1-5점
	2. 컨텍스트 관련성 (Context Relevance) : 1-5점
	3. 컨텍스트 충실성 (Context Faithfulness) : 1-5점
	4. 컨텍스트 충분성 (Context Recall) : 1-5점
	5. total_score : 총점
	6. analysis: 종합적인 분석 의견

In [9]:
import pandas as pd
import json
import os
from tqdm import tqdm
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def evaluate_rag_with_llm(data_df: pd.DataFrame):
    """
    LLM 평가 메트릭으로 RAG 시스템 평가
    data_df columns: ['user_input','retrieved_contexts','response','reference']
    """
    user_inputs = data_df['user_input'].tolist()
    retrieved_contexts = data_df['retrieved_contexts'].tolist()
    responses = data_df['response'].tolist()
    references = data_df['reference'].tolist()

    results = []

    prompt_template = """당신은 Google API 문서 기반 RAG 챗봇의 평가자입니다.
점수는 1~5 정수이며, 아래 JSON 스키마로만 출력하세요.

[입력]
- 질문: {user_input}
- 검색된 컨텍스트: {retrieved_context}
- 모델 응답: {response}
- 참조/기대 답변: {reference}

[평가기준 및 점수 레벨]

1) answer_correctness (정답 정확성: API 계약·권장사항 관점)
- 5: 제품/메서드/경로/필수 파라미터/스코프/버전·Deprecated/쿼터·요금·리전 제약까지 모두 정확
- 4: 핵심 정확, 사소한 누락/표현 오류만 존재(위험·오류 유발 없음)
- 3: 핵심은 맞으나 의미 있는 세부 요소 1개 이상 오류 또는 최신성 불명확
- 2: 서비스·메서드·경로·스코프 중 하나 이상 틀림, 위험하거나 오해 소지 큰 조언
- 1: 존재하지 않는 API/메서드/스코프 주장 등 근본적 오류

2) groundedness (근거 충실성: 컨텍스트 기반 여부)
- 5: 핵심 주장·식별자 모두 컨텍스트로 확인 가능, 필요 시 직인용 가능
- 4: 거의 전부 근거 기반, 비핵심 보충만 컨텍스트 밖
- 3: 근거와 비근거 혼재, 핵심 요소 1개 이상이 컨텍스트에 없음
- 2: 대부분 컨텍스트 밖 주장 의존, 용어만 차용
- 1: 컨텍스트와 모순되거나 없는 요소를 단정

3) context_coverage (컨텍스트 충분성: 검색 품질)
- 5: 메서드 개요, 필수 파라미터, 권한/스코프, 핵심 제약(쿼터/요금/리전/오류) 모두 포함
- 4: 대부분 포함, 부가적 세부 1개 정도만 부족
- 3: 핵심 구성 1–2개 부족(스코프/필수 파라미터 등), 추정 필요
- 2: 핵심 다수 누락, 컨텍스트만으론 신뢰도 낮음
- 1: 무관/빈약하여 답 도출 불가

[출력: JSON만]
{{
  "answer_correctness": 1,
  "groundedness": 1,
  "context_coverage": 1,
  "analysis": "무엇이 맞고/틀렸는지, 점수 근거를 간단히"
}}
다른 형식의 응답은 하지 마세요. 오직 마크다운이 아닌 JSON만 반환하세요."""

    for i in tqdm(range(len(user_inputs)), total=len(user_inputs), desc="RAG 평가 진행 중"):
        try:
            prompt = prompt_template.format(
                user_input=user_inputs[i],
                retrieved_context=retrieved_contexts[i],
                response=responses[i],
                reference=references[i]
            )

            resp = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "당신은 RAG 평가 도구입니다. 반드시 유효한 JSON 형식으로만 응답하세요."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0,
                response_format={"type": "json_object"}
            )

            # JSON 파싱 + 숫자 강제
            parsed = json.loads(resp.choices[0].message.content)
            answer_correctness = int(parsed.get('answer_correctness', 1))
            answer_groundedness = int(parsed.get('groundedness', 1))          # = context faithfulness
            context_coverage   = int(parsed.get('context_coverage', 1))
            analysis           = parsed.get('analysis', "파싱 오류 또는 결과 누락")

            total_score = answer_correctness + answer_groundedness + context_coverage

            results.append({
                'id': i,
                'user_input': user_inputs[i],
                'answer_correctness': answer_correctness,
                'answer_groundedness': answer_groundedness,
                'context_coverage': context_coverage,
                'total_score': total_score,
                'analysis': analysis
            })

        except Exception as e:
            print(f"항목 {i} 평가 중 오류 발생: {e}")
            results.append({
                'id': i,
                'user_input': user_inputs[i],
                'error': str(e)
            })

    results_df = pd.DataFrame(results)

    # 숫자 컬럼 보정(혹시 문자열이 섞였을 경우 대비)
    for col in ['answer_correctness', 'answer_groundedness', 'context_coverage', 'total_score']:
        if col in results_df.columns:
            results_df[col] = pd.to_numeric(results_df[col], errors='coerce')

    metrics_summary = None
    if 'total_score' in results_df.columns and results_df['total_score'].notna().any():
        # 1~5 평균을 100점 환산(×20)
        per_row_avg_5 = results_df[['answer_correctness','answer_groundedness','context_coverage']].mean(axis=1)
        overall_100   = per_row_avg_5.mean() * 20.0
        correctness_100 = results_df['answer_correctness'].mean() * 20.0
        grounded_100    = results_df['answer_groundedness'].mean() * 20.0
        coverage_100    = results_df['context_coverage'].mean() * 20.0

        metrics_summary = {
            '전체 평균(100)': overall_100,
            '정답 정확성 평균(100)': correctness_100,
            '근거 충실성 평균(100)': grounded_100,
            '컨텍스트 충분성 평균(100)': coverage_100
        }

        print("\n===== 평가 요약 =====")
        for metric, value in metrics_summary.items():
            print(f"{metric}: {value:.2f}")

    return (results_df, metrics_summary) if metrics_summary is not None else results_df



In [10]:
v7_results_df, v7_metrics_summary = evaluate_rag_with_llm(v7_evaluated_dataset)

RAG 평가 진행 중: 100%|██████████| 20/20 [01:02<00:00,  3.15s/it]


===== 평가 요약 =====
전체 평균(100): 70.67
정답 정확성 평균(100): 70.00
근거 충실성 평균(100): 72.00
컨텍스트 충분성 평균(100): 70.00


In [11]:
v7_results_df

,id,user_input,answer_correctness,answer_groundedness,context_coverage,total_score,analysis
0,0,CSV 파일을 이용해서 S/MIME 인증서를 관리하려면 어떤 작업이 필요한가요?,4,4,4,12,"모델의 응답은 S/MIME 인증서를 관리하기 위한 CSV 파일 준비, 처리 함수 작..."
1,1,클라우드 파이어스토어 부하 분산기 어떻게 써야되나요?,4,4,4,12,모델 응답은 Cloud Firestore의 자동 확장 기능에 대한 핵심 내용을 정확...
2,2,범위 많으면 동의 어렵나요?,1,1,1,3,"모델 응답은 질문에 대한 적절한 답변을 제공하지 않으며, 관련된 정보가 전혀 포함되..."
3,3,As a beginner who is not familiar with Google ...,4,4,4,12,모델 응답은 세션 제어 기능이 GCP 리소스 접근에 미치는 영향을 잘 설명하고 있으...
4,4,"firebase_functions 어떻게 쓰는지 잘 모르겠는데, Cloud Fire...",5,5,5,15,모델의 응답은 Cloud Firestore에서 문서 변경 시 자동으로 실행되는 기능...
5,5,Firebase에서 다단계 인증(MFA)을 사용할 때 Objective-C로 사용자...,4,4,4,12,"모델의 답변은 다단계 인증을 처리하는 방법에 대한 핵심적인 내용을 포함하고 있으며,..."
6,6,Java를 사용해서 Firestore에 문서를 추가할 때 어떤 코드를 사용하나요?,3,4,3,10,모델의 응답은 Firestore에 문서를 추가하는 방법에 대한 기본적인 코드 예시를...
7,7,COLUMNS가 Dimension API에서 무슨 뜻인가요?,3,3,3,9,"모델 응답은 COLUMNS의 의미를 언급했지만, 세부 사항이 부족하고 일부 중요한 ..."
8,8,ttll이 뭐에요? 그리고 한도 같은거 있나요?,1,1,1,3,"모델의 응답은 질문에 대한 정확한 정보가 없으며, ttll과 한도에 대한 구체적인 ..."
9,9,Me want know what FUNCTION_ENTRYPOINT do in gc...,1,1,1,3,모델의 응답은 'FUNCTION_ENTRYPOINT'에 대한 정보가 없다고 주장했으...


In [12]:
v7_results_df.to_csv('[5.utils_v6]v7_evaluation_results.csv', index=False)

In [13]:
v7_metrics_summary

{'전체 평균(100)': np.float64(70.66666666666667),
 '정답 정확성 평균(100)': np.float64(70.0),
 '근거 충실성 평균(100)': np.float64(72.0),
 '컨텍스트 충분성 평균(100)': np.float64(70.0)}